In [ ]:
%run QM9GNN2_Multitask.ipynb

In [ ]:
import itertools
import pandas as pd
import pickle

from sklearn.cluster import SpectralClustering

In [ ]:
def generate_single_task_filename(task):
    return path.join('single_task_trained_models', f'{task}.h5')

In [ ]:
A_all, X_all, E_all, y_all = load_data()
A, X, E, y = sample_from_data(10000, A_all, X_all, E_all, y_all)
_ = standardize(y)
A_train, A_test, \
    X_train, X_test, \
    E_train, E_test, \
    y_train, y_test = train_test_split(A, X, E, y, test_size=0.1)
tasks = [t for c in clusters for t in c]

In [ ]:
def train_and_save_single_task_models(tasks, *, A, X, E):
    for task in tasks:
        print(f'learning {task}')
        model = build_single_task_model(A=A, X=X, E=E)
        model.fit([X_train, A_train, E_train],
                 y_train[[task]].values,
                 batch_size=32,
                 epochs=5)
        model.save_weights(generate_single_task_filename(task))

In [ ]:
def calculate_transfer_coefficient(learned, transferred, y_transferred_test, A_test, X_test, E_test):
    learned_model = build_single_task_model(A=A_test, X=X_test, E=E_test)
    transferred_model = build_single_task_model(A=A_test, X=X_test, E=E_test)
    learned_model.load_weights(generate_single_task_filename(learned))
    transferred_model.load_weights(generate_single_task_filename(transferred))
    learned_layers = learned_model.get_weights()
    transferred_layers = transferred_model.get_weights()
    # 10 is a hard-coded architecture-dependent value
    SPLIT = -2
    transferred_layers = learned_layers[:SPLIT] + transferred_layers[SPLIT:]
    transferred_model.set_weights(transferred_layers)
    transferred_model.compile(optimizer=Adam(lr=1e-3), loss='mae')
    
    eval_results = transferred_model.evaluate([X_test, A_test, E_test],
                                             y_transferred_test,
                                              batch_size=32)
    return eval_results

In [ ]:
def save_transfer_coefficients(tasks, A_test, X_test, E_test, y_test):
    df = pd.DataFrame(index=tasks, columns=tasks, dtype=np.float32)
    for learned, transferred in itertools.product(tasks[:2], repeat=2):
        print(f'transferring {learned} to {transferred}')
        transfer_coef = calculate_transfer_coefficient(learned, 
                                                       transferred, 
                                                       y_test[[transferred]].values, 
                                                       A_test, 
                                                       X_test, 
                                                       E_test)
        print(transfer_coef)
        df.loc[learned, transferred] = transfer_coef

    with open('cross_task_transfer_coefs.pkl', 'wb') as f:
        pickle.dump(obj=df, file=f)
    
    return df

In [ ]:
def load_transfer_coefficients():
    with open('cross_task_transfer_coefs.pkl', 'rb') as f:
        df = pickle.load(f)
        return df

In [ ]:
def cluster_from_similarity_matrix(df, num_clusters):
    mat = df.to_numpy()
    df[:] = (mat+mat.T)/2
    sc = SpectralClustering(n_clusters=num_clusters, affinity='precomputed')
    labels = sc.fit_predict(df.to_numpy())
    d = {c:[] for c in labels}
    for c, t in zip(labels, df.columns):
        d[c].append(t)
    clusters = [v for k, v in d.items()]
    return clusters

In [ ]:
train_and_save_single_task_models(tasks, A=A_train, X=X_train, E=E_train)

In [ ]:
save_transfer_coefficients(tasks, A_test, X_test, E_test, y_test)

In [ ]:
df = load_transfer_coefficients()
print(cluster_from_similarity_matrix(df, 6))